In [3]:
from dask_cassandra_loader import Loader

dask_cassandra_loader = Loader()


###  cassandra ornegi çalıtırma

wget https://dask-cassandra-loader.readthedocs.io/en/latest/_downloads/5f6a89aeb41f1a7e7bdcb0b51bd37a76/tutorial.cql

docker run --name cassandra-server -p 9042:9042 -d -v bitcassvol:/bitnami bitnami/cassandra:latest

docker run --name castest -it --rm bitnami/cassandra:latest bash

#### hostta
docker cp tutorial.cql castest:/

#### castest container'ında
HOST_IP=10.151.255.101
cqlsh -u cassandra -p cassandra $HOST_IP -e "create keyspace tutorial with replication = {'class': 'SimpleStrategy', 'replication_factor': 1};"

cqlsh -u cassandra -p cassandra $HOST_IP --keyspace=tutorial -f tutorial.cql


In [1]:
HOST_IP="10.151.255.101"
keyspace = 'tutorial'
cluster = [HOST_IP]

In [4]:
dask_cassandra_loader.connect_to_cassandra(cluster,
                                           keyspace,
                                           username='cassandra',
                                           password='cassandra')

In [5]:
dask_cassandra_loader.connect_to_local_dask()

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: inproc://10.151.255.101/16600/1
distributed.scheduler - INFO -   dashboard at:            localhost:8787
distributed.worker - INFO -       Start worker at: inproc://10.151.255.101/16600/3
distributed.worker - INFO -          Listening to:       inproc10.151.255.101
distributed.worker - INFO -          dashboard at:       10.151.255.101:43609
distributed.worker - INFO - Waiting to connect to: inproc://10.151.255.101/16600/1
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -               Threads:                         32
distributed.worker - INFO -                Memory:                  134.85 GB
distributed.worker - INFO -       Local Directory: /home/dev/Practice/myjupyter/course/dask-worker-space/worker-ty1f2lih
distributed.worker - INFO - -------------------------------------------------
distributed.scheduler - INFO - Register worke

In [6]:
table = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],
                                         [('day', 'equal', [8])],
                                         [('id', [18]), ('year', [2018]),
                                          ('month', [11])],
                                         force=False)

In [7]:
if table is None:
    raise AssertionError("Table is not supposed to be None!!!")

if table.data is None:
    raise AssertionError("Table.data is not supposed to be None!!!")

In [8]:
local_table = table.data.compute()

In [9]:
print(local_table.head())

   id  year  month  day
0  18  2018     11    8


In [10]:
table2 = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],[],[],
                                         force=True)

In [11]:
local_table2 = table2.data.compute()

In [12]:
local_table2.head()

,id,year,month,day
0,18,2018,12,8
0,18,2018,11,8
